In [5]:
import torch
import matplotlib.pyplot as plt
import torchvision
import skimage
from skimage.measure import compare_ssim
import torchvision.transforms as transforms
import numpy as np
import time
from PIL import Image
import scipy.ndimage as ndimage
import torch.nn as nn
import os
from skimage import io,exposure,img_as_ubyte
import glob
import torchvision.transforms as transforms
import argparse
from models import GetModel

In [6]:
opt = argparse.Namespace()

opt.root = 'Test_data'
opt.out = 'test_output'
opt.task = 'simin_gtout'
opt.norm = 'minmax'
opt.dataset = 'fouriersim'

opt.model = 'rcan'

# data
opt.imageSize = 512
opt.weights = 'model_output/final.pth'

# input/output layer options
opt.scale = 1
opt.nch_in = 9
opt.nch_out = 1

# architecture options 
opt.narch = 0
opt.n_resblocks = 3
opt.n_resgroups = 5
opt.reduction = 16
opt.n_feats = 48

# test options
opt.test = False
opt.cpu = False
opt.device = torch.device('cuda' if torch.cuda.is_available() and not opt.cpu else 'cpu')

In [12]:
def LoadModel(opt):
    print('Loading model')
    print(opt)

    net = GetModel(opt)
    print('loading checkpoint',opt.weights)
    checkpoint = torch.load(opt.weights)

    if type(checkpoint) is dict:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint

    net.load_state_dict(state_dict)

    return net


def SIM_reconstruct(model, opt):
    
    def prepimg(stack,self):

        inputimg = stack[:9]

        if self.nch_in == 6:
            inputimg = inputimg[[0,1,3,4,6,7]]
        elif self.nch_in == 3:
            inputimg = inputimg[[0,4,8]]

        if inputimg.shape[1] > 512 or inputimg.shape[2] > 512:
            print('Over 512x512! Cropping')
            inputimg = inputimg[:,:512,:512]

        inputimg = inputimg.astype('float') / np.max(inputimg) # used to be /255
        widefield = np.mean(inputimg,0) 

        if self.norm == 'adapthist':
            for i in range(len(inputimg)):
                inputimg[i] = exposure.equalize_adapthist(inputimg[i],clip_limit=0.001)
            widefield = exposure.equalize_adapthist(widefield,clip_limit=0.001)
            inputimg = torch.from_numpy(inputimg).float()
            widefield = torch.from_numpy(widefield).float()
        else:
            # normalise 
            inputimg = torch.from_numpy(inputimg).float()
            widefield = torch.from_numpy(widefield).float()
            widefield = (widefield - torch.min(widefield)) / (torch.max(widefield) - torch.min(widefield))

            if self.norm == 'minmax':
                for i in range(len(inputimg)):
                    inputimg[i] = (inputimg[i] - torch.min(inputimg[i])) / (torch.max(inputimg[i]) - torch.min(inputimg[i]))

        return inputimg,widefield

    os.makedirs('%s' % opt.out,exist_ok=True)
    files = glob.glob('%s/*.tif' % opt.root)
    
    for iidx,imgfile in enumerate(files):
        
        print('[%d/%d] Reconstructing %s' % (iidx+1,len(files),imgfile))
        stack = io.imread(imgfile)
        
        inputimg, wf = prepimg(stack,opt)
        wf = (255*wf.numpy()).astype('uint8')

        with torch.no_grad():
            sr = model(inputimg.unsqueeze(0).to(opt.device))
            sr = sr.cpu()
            sr = torch.clamp(sr,min=0,max=1) 

        sr = sr.squeeze().numpy()
        sr = (255*sr).astype('uint8')
        if opt.norm == 'adapthist':
            sr = exposure.equalize_adapthist(sr,clip_limit=0.01)

        skimage.io.imsave('%s/test_wf_%d.jpg' % (opt.out,iidx), wf)
        skimage.io.imsave('%s/test_sr_%d.jpg' % (opt.out,iidx), sr) 

In [13]:
net = LoadModel(opt)
SIM_reconstruct(net,opt)

Loading model
Namespace(cpu=False, dataset='fouriersim', device=device(type='cuda'), imageSize=512, model='rcan', n_feats=48, n_resblocks=3, n_resgroups=5, narch=0, nch_in=9, nch_out=1, norm='minmax', out='test_output', reduction=16, root='Test_data', scale=1, task='simin_gtout', test=False, weights='model_output/final.pth')
loading checkpoint model_output/final.pth
[1/6] Reconstructing Test_data\AtheiSIM 13-43_488.tif
[2/6] Reconstructing Test_data\AtheiSIM 14-52_488.tif
[3/6] Reconstructing Test_data\barbara.tif
[4/6] Reconstructing Test_data\penguin.tif
[5/6] Reconstructing Test_data\SLM-SIM beads.tif
[6/6] Reconstructing Test_data\SLM-SIM syrlyso640-5.tif


### 